In [88]:
import cv2
from os import listdir
from os.path import join
import numpy as np
import pandas as pd
from sklearn import svm

In [89]:
# Listing all the image files

mypath = '/home/rohan/Desktop/CAVIAR4REID/archive/'
files = listdir(mypath)
del files[-1]

In [90]:
# Building a list that groups ALL THE VIEWS from DIFFERENT ANGLES of the SAME PERSON

person = []
k = listdir(mypath)

for i in range(len(files)):
    j = '00'+str(i+1)+'_' #To identify all files in the form 1_,2_,3_ etc
    k = '000'+str(i+1)+'_' #To identify all files in the form 1_,2_,3_ etc
    person_num = [f.startswith((j,k)) for f in files] #Boolean values of all places very j(or '(i+1)_') is True
    idx =  [k for k, x in enumerate(person_num) if x] #Return indices of where the values are True
    (person.append([files[l] for l in idx]))
person = [x for x in person if x != []]
# person

In [91]:
#Counting the number of views of each person

mypath = '/home/rohan/Desktop/CAVIAR4REID/archive/'
count = np.empty((len(person)),dtype=object)
for i in range(len(person)):
    count[i] = len(person[i])

In [92]:
# Specifying path for each image

person = np.array(person)
person_path = []
mypath = '/home/rohan/Desktop/CAVIAR4REID/archive/'
for p in person:
    each_p = p
    for i in each_p:
        i = mypath + i
        (person_path).append(i)

In [94]:
# Reading and resizing images

images = np.empty((len(person_path)), dtype=object)
resized_imgs = np.empty((len(person_path)), dtype=object)
for n in range(0, len(person_path)):
    images[n] = cv2.imread(person_path[n],0) #Reading images in grayscale
    resized_imgs[n] = cv2.resize(images[n],(32,32)) #Resizing images to 32x32 standard size
# images

In [95]:
# Importing cleaned labels from csv file

labels = pd.read_csv(filepath_or_buffer = r'/home/rohan/Desktop/Minor/labelled_caviar.csv',index_col = 0)

In [96]:
labels.head()

,Upper Body Colour,Lower Body Colour,Hair Colour,Footwear Colour,Lower Body Type,Lower Body Clothing,Gender,Age Limit,Upper Body Type,Upper Body Clothing,Hair Length,Footear Type,carrying,Accessories
1,9,0,0,0,0,3,1,2,0,0,0,2,0,0
2,0,0,0,0,0,0,0,0,1,1,0,0,1,0
3,0,0,0,0,1,0,0,1,1,2,0,1,2,0
4,1,1,0,0,0,0,0,1,1,3,0,1,2,0
5,5,2,0,0,0,1,0,1,1,4,0,0,2,0


In [97]:
# Expanding labels to match with the number of images according to count

extended_labels = []
for i in range(len(labels)):
    extended_labels.append(np.tile(np.array(pd.DataFrame(labels.iloc[i,:]).transpose()),(count[i],1)))
    #Copying labels as per the count values
# extended_labels

In [98]:
# Associating each label with each training sample

final_labels = []
for i in range(len(extended_labels)):
    for j in range(len(extended_labels[i])):
            final_labels.append(extended_labels[i][j])
# csv = pd.DataFrame(final_labels)
# csv.to_excel(r'C:\Users\user\Desktop\Labels.xlsx')
# final_labels

In [99]:
mypath = '/home/rohan/Desktop/CAVIAR4REID/archive/'
k = listdir(mypath)
img = cv2.imread(join(mypath,k[0]))
img = cv2.resize(img,(32,32))
len(k)

1220

In [100]:
# Defining X_train and y_train

mypath = '/home/rohan/Desktop/CAVIAR4REID/archive/'
k = listdir(mypath)
images = np.empty((len(k)-1),dtype=object)
resized_imgs = np.empty((len(k)-1),dtype=object)
im_list = []
for n in range(0, len(k)-1):
    images[n] = cv2.imread(join(mypath,k[n]),1) #Reading images in colour
    resized_imgs[n] = cv2.resize(images[n],(32,32)) #Resizing images to 32x32 standard size
    im_list.append(resized_imgs[n])
  
im_list = np.array(im_list)
a=im_list.mean()
b=im_list.std()

for i in range(0,len(k)-1):
    im_list[i]=(im_list[i]-a)/b
    
X_train = im_list
y_train = final_labels

In [116]:
# Reshaping X_train to make it compatible with theano backend
X_train = np.swapaxes(X_train,1,3)
X_train.shape

(1219, 3, 32, 32)

In [15]:
# Flattening X_train (only for SVM and KNN)

# for i in range(len(k)-1):
#     X_train[i] = X_train[i].flatten().tolist()

In [103]:
#Flattening y_train

# y_train = np.array(y_train)
for i in range(len(k)-1):
    y_train[i] = y_train[i].tolist()

In [105]:
X_train = np.array(list(X_train), dtype=np.float)

In [106]:
# CNN
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [107]:
# Converts classes to categorical values for every attribute

def categorical_classes(y_train,classes,att_no,k=len(y_train)):
    att = []
    for i in range(0,k):
        ind = y_train[i][att_no]
        att.append(ind)
    att = np.array(att,dtype=int)
    att = np_utils.to_categorical(att, classes)
    return (att)

In [155]:
# Collecting all these values in one array

classes_arr = (18,6,3,4,2,6,2,4,2,21,2,6,10,5) # Number of classes for each attribute
iter = (0,1,2,3,4,5,6,7,8,9,10,11,12,13)
arr = []
for i,j in zip(classes_arr,iter):
    temp = []
    temp = categorical_classes(y_train,i,j)
    arr.append(temp)
    
arr[0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [183]:
batch_size = 128
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 32, 32
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
# input shape when using Theano as backend
input_shape = (3, img_rows, img_cols)

model = []

for i in (classes_arr):
    
    nb_classes = i
    
    temp_model = Sequential()

    temp_model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                            border_mode='valid',
                            input_shape=input_shape))
    temp_model.add(Activation('relu'))
    temp_model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
    temp_model.add(Activation('relu'))
    temp_model.add(MaxPooling2D(pool_size=pool_size))
    temp_model.add(Dropout(0.25))

    temp_model.add(Flatten())
    temp_model.add(Dense(128))
    temp_model.add(Activation('relu'))
    temp_model.add(Dropout(0.5))
    temp_model.add(Dense(nb_classes))
    temp_model.add(Activation('softmax'))

    temp_model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    model.append(temp_model)
len(model)

14

In [122]:
# TESTING

# Reading X_test and normalizing

testpath = '/home/rohan/Desktop/Test Set'
l = listdir(testpath)
test_images = np.empty((len(l)-1),dtype=object)
test_resized_imgs = np.empty((len(l)-1),dtype=object)
im_list_test = []
for n in range(0, len(l)-1):
    test_images[n] = cv2.imread(join(testpath,l[n]))
    test_resized_imgs[n] = cv2.resize(test_images[n],(32,32)) #Resizing images to 32x32 standard size
    im_list_test.append(test_resized_imgs[n])

im_list_test = np.array(im_list_test)
a_test=im_list_test.mean()
b_test=im_list_test.std()

for i in range(0,len(l)-1):
    im_list_test[i]=(im_list_test[i]-a_test)/b_test
    
X_test = im_list_test
X_test = np.swapaxes(X_test,1,3)

In [178]:
# Y_test labels
import csv
with open('/home/rohan/Downloads/labelled_test_ViPer.csv', 'rb') as csvfile:
     Y_test = csv.reader(csvfile)
     Y_test = list(Y_test)
     Y_test = np.array(Y_test,dtype=np.uint8)
Y_test[7][6] = 0

In [184]:
%%time
for i,j in zip(range(0,len(model)),classes_arr):
    
    model[i].fit(X_train,arr[i], batch_size=batch_size, nb_epoch=nb_epoch)
    score = model[i].evaluate(X_test, np_utils.to_categorical(Y_test.T[i],j), verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Epoch 1/12
1219/1219 [==============================] - 6s - loss: 11.4222 - acc: 0.2756     
Epoch 2/12
1219/1219 [==============================] - 7s - loss: 10.3071 - acc: 0.3568     
Epoch 3/12
1219/1219 [==============================] - 6s - loss: 10.3066 - acc: 0.3568     
Epoch 4/12
1219/1219 [==============================] - 7s - loss: 10.2668 - acc: 0.3601     
Epoch 5/12
1219/1219 [==============================] - 7s - loss: 10.1888 - acc: 0.3626     
Epoch 6/12
1219/1219 [==============================] - 6s - loss: 10.2004 - acc: 0.3618     
Epoch 7/12
1219/1219 [==============================] - 6s - loss: 10.2432 - acc: 0.3568     
Epoch 8/12
1219/1219 [==============================] - 6s - loss: 10.2054 - acc: 0.3610     
Epoch 9/12
1219/1219 [==============================] - 7s - loss: 10.2201 - acc: 0.3626     
Epoch 10/12
1219/1219 [==============================] - 7s - loss: 10.2081 - acc: 0.3634     
Epoch 11/12
1219/1219 [==============================] - 7s

In [215]:
# TESTING

# Reading X_test and normalizing

testpath = '/home/rohan/Desktop/Test Set'
l = listdir(testpath)
test_images = np.empty((len(l)-1),dtype=object)
test_resized_imgs = np.empty((len(l)-1),dtype=object)
im_list_test = []
for n in range(0, len(l)-1):
    test_images[n] = cv2.imread(join(testpath,l[n]),0) #Reading images in grayscale
    test_resized_imgs[n] = cv2.resize(test_images[n],(32,32)) #Resizing images to 32x32 standard size
    im_list_test.append(test_resized_imgs[n])

im_list_test = np.array(im_list_test)
a_test=im_list_test.mean()
b_test=im_list_test.std()

for i in range(0,len(l)-1):
    im_list_test[i]=(im_list_test[i]-a_test)/b_test
    
X_test = im_list_test

array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ..., 255,   0,   0],
        [  0,   0,   0, ..., 255,   0,   0],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   1, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0, 255,   0],
        ..., 
        [  0,   0,   0, ...,   1,   1,   1],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [255, 255,   0, ...,   0,   0,   0],
        ..., 
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       ..., 
       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   1],
        [  0,   0,   0, ...,   0,   0, 

In [53]:
%%time
# Training an SVM

trained_SVM = []
clf=svm.SVC(gamma=0.001)
m,n = y_train.shape

Wall time: 2 ms


In [41]:
# TESTING

In [42]:
# Reading X_test and normalizing

testpath = 'C:/Users/user/Desktop/Minor/Test Set'
l = listdir(testpath)
test_images = np.empty((len(l)-1),dtype=object)
test_resized_imgs = np.empty((len(l)-1),dtype=object)
for n in range(0, len(l)-1):
    test_images[n] = cv2.imread(join(testpath,l[n]),0) #Reading images in grayscale
    test_resized_imgs[n] = cv2.resize(test_images[n],(32,32)) #Resizing images to 32x32 standard size
for i in range(0,len(l)-1):
    # Normalizing raw pixel values using mean and standard deviation
    mean=test_resized_imgs[i].mean()
    std=test_resized_imgs[i].std()
    t=np.ndarray([32,32])
    t.fill(1)
    t=t*mean
    test_resized_imgs[i]=(test_resized_imgs[i]-mean)/std
    
X_test = test_resized_imgs

In [43]:
# Flattening X_test

X_test = np.array(X_test)

for i in range(len(X_test)):
    X_test[i] = X_test[i].flatten().tolist()
X_test = np.array(list(X_test), dtype=np.float)

In [44]:
%%time
# Predicting

predictions = np.empty((n,len(X_test)))
for i in range(len(X_test)):
    for j in range(0,n):
        clf.fit(X_train,y_train[:,j])
        predictions[j][i] = clf.predict(X_test[i])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C

Wall time: 7min 9s


C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [45]:
predictions

array([[ 1.,  0.,  0.,  1.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  1.,  1.,
         0.,  0.],
       [ 0.,  2.,  2.,  2.,  2.,  2.,  0.,  5.,  2.,  2.,  2.,  2.,  2.,
         2.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.],
       [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
         1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.],
       [ 6.,  6.,  4.,  1.,  6.,  4.,  6.,  3.,  3.,  3.,  4.,  6.,  3.,
         4.,  6.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       

In [ ]:
# KNN

In [138]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train, y_train)
print(knn.predict(X_test))

[[0 0 0 0 1 0 1 1 0 1 0 0 0 0]
 [2 2 0 0 0 0 0 0 1 4 0 3 2 0]
 [0 5 0 0 0 0 1 0 1 3 0 0 2 0]
 [1 2 0 0 1 1 0 1 1 1 0 0 2 0]
 [1 0 0 0 1 0 1 1 0 1 0 0 2 0]
 [0 2 0 0 1 2 0 1 0 9 0 1 2 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 2 0]
 [2 5 0 0 0 0 0 1 1 4 0 3 2 0]
 [0 5 0 0 0 0 0 1 1 4 0 0 2 0]
 [0 2 0 1 0 1 1 1 1 6 1 0 2 0]
 [0 0 0 0 0 0 0 1 1 3 0 0 2 0]
 [1 2 0 0 1 2 0 1 0 1 0 1 2 0]
 [1 2 0 0 0 0 0 1 1 1 0 1 2 0]
 [0 2 0 0 0 1 0 1 1 9 0 0 2 0]
 [0 0 0 0 0 1 0 1 1 4 0 0 2 0]]
Wall time: 133 ms
